<a href="https://colab.research.google.com/github/areddysathvik/SPAM-CLASSIFICATION/blob/main/Roberta-SpamClassfication-SimpleTransformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
path = "/SPAM text message 20170820 - Data(1).csv"

In [31]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd

df = pd.read_csv(path)
df.columns = ['labels','text']

In [33]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [46]:
import re
import nltk
nltk.download('stopwords')
nltk.download('corpus')
nltk.download('wordnet')

s_words = list(set(stopwords.words('english')))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
  text = text.lower()
  text = re.sub('[^a-zA-Z0-9]',' ',text)
  text = [lemmatizer.lemmatize(x) for x in word_tokenize(text) if x not in s_words and len(x) > 2]
  return " ".join(text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading corpus: Package 'corpus' not found in index
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [47]:
X_train['text'] = X_train['text'].apply(preprocess)

In [49]:
model_args = ClassificationArgs()
model_args.num_train_epochs=5
model_args.overwrite_output_dir=True
model_args.labels_list = ["spam", "ham"]

X_train = df[:round(0.7*(df.shape[0]))]
X_test = df[round(0.7*(df.shape[0]))+1:]

model = ClassificationModel(
    "roberta", "roberta-base", args=model_args,use_cuda=True,
)

# Train the model
model.train_model(X_train)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should pr

  0%|          | 0/3900 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/488 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/488 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/488 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/488 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/488 [00:00<?, ?it/s]

  0%|          | 0/1671 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/209 [00:00<?, ?it/s]

In [51]:
X_test['text'] = X_test['text'].apply(preprocess)

result, model_outputs, wrong_predictions = model.eval_model(X_test)

<ipython-input-51-8f24a53b7d4c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['text'] = X_test['text'].apply(preprocess)


  0%|          | 0/1671 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/209 [00:00<?, ?it/s]

In [52]:
len(wrong_predictions)

28

In [54]:
result

{'mcc': 0.9282080677781748,
 'tp': 1432,
 'tn': 211,
 'fp': 17,
 'fn': 11,
 'auroc': 0.9938617767565137,
 'auprc': 0.9987165620501111,
 'eval_loss': 0.09742024842383447}

In [55]:
model_outputs

array([[-3.99414062,  3.9296875 ],
       [-3.99414062,  3.93164062],
       [-4.        ,  3.9375    ],
       ...,
       [-3.99414062,  3.93164062],
       [-3.9921875 ,  3.92578125],
       [-3.98828125,  3.91796875]])